<a href="https://colab.research.google.com/github/shivendr7/ml/blob/GANs/ModelGAN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi, in this lesson you will discover how to implement a simple discriminator and generator model using the Keras deep learning library.

We will assume the images in our domain are 28x28 pixels in size and color, meaning they have three color channels.

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam

Discriminator Model
The discriminator model accepts an image with the with size 28x28x3 pixels and must classify it as real (1) or fake (0) via the sigmoid activation function.

Our model has two convolutional layers with 64 filters each and uses same padding. Each convolutional layer will downsample the input using a 2x2 stride, which is a best practice for GANs, instead of using a pooling layer.

Also following best practice, the convolutional layers are followed by a LeakyReLU activation with a slope of 0.2 and a batch normalization layer.

In [6]:
def build_discriminator():
  model= Sequential()

  #downsample to 14x14
  model.add(Conv2D(64,(3,3), strides=(2,2), padding='same', input_shape=(28,28,3)))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  
  #downsample to 7x7
  model.add(Conv2D(64,(3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  
  #classify
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

In [7]:
build_discriminator()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

Generator Model
The generator model takes a 100-dimensional point in the latent space as input and generates a 28x28x3.

The point in latent space is a vector of Gaussian random numbers. This is projected using a Dense layer to the basis of 64 tiny 7x7 images. The small images are then upsampled twice using two transpose convolutional layers with a 2x2 stride and followed by a LeakyReLU and BatchNormalization layers, which are a best practice for GANs.

The output is a three channel image with pixel values in the range [-1,1] via the Tanh activation function.

In [9]:
def build_generator():
  model= Sequential()

  #formation for 7x7 image
  n_nodes= 64*7*7
  model.add(Dense(n_nodes, input_dim=100))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(Reshape((7,7,64)))

  #upsampling to 14x14
  model.add(Conv2DTranspose(64, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())

  #upsampling to 28x28
  model.add(Conv2DTranspose(64, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

  model.summary()

In [10]:
build_generator()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3136)              316736    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 3136)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3136)              12544     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 64)       